In [2]:
import pdfplumber
from pathlib import Path
import re
import numpy as np
import fitz

In [3]:
from preprocessing.IFRS import *

In [133]:
#First assume every page contains only structured texts with one column 

tables = []
text_blocks = []
words = []

with pdfplumber.open("data/raw/Bale_3.pdf") as pdf:
    for page in pdf.pages:
        text_blocks.append(page.extract_text())
        tables.append(page.extract_tables()) #Will help identify page with table to perform specific cleaning
        words.append(page.extract_words())

In [172]:
def extract_text_from_bale_lines(pdf) :

    texts_ = []
    
    for i in pdf :
        global_ = []

        for j in range(0,len(i)) :

            word = i[j]
            word_1 = i[j-1]

            if word_1["height"] == 6 :
                word_1 = i[j-2]

            word["height"] = round(word["height"],1)
            word_1["height"] = round(word_1["height"],1)

            #do not count footpage/headers 
            if (word["top"] > 780) or (word["height"] >15):
                continue

            if global_ == [] : 
                if word["height"] >= 12 :
                    global_.extend(f'_title_{word["text"]}')
#                elif word["height"] > 9 :
#                    global_.extend(f'_subtitle_{word["text"]}')
                else : 
                    global_.extend(word["text"])

            elif (word["top"] == word_1["top"]) : #Same line same part
                global_.extend(word["text"])

            # Between inf then same part
            elif abs(word_1["top"] - word["top"]) < 13 : 
                global_.extend(word["text"])     

            elif abs(word_1["top"] - word["top"]) > 13 : 
#                global_.extend(f'\n{word["text"]}')

                if (word["height"] >= 13) :
                    global_.extend(f'\n_title_{word["text"]}')
#                elif word["height"] > 9 :
#                    global_.extend(f'\n_subtitle_{word["text"]}')
                elif (word["height"] < 8) :
                    global_.extend(f'\n_footnote_{word["text"]}')
                
#                elif (word_1["height"] == 11) :
#                    global_.extend(f'_title_\n{word["text"]}')
#                elif word_1["height"] >= 9 :
#                    global_.extend(f'_subtitle_\n{word["text"]}')
#                elif (word_1["height"] < 8) :
#                    global_.extend(f'_footnote_\n{word["text"]}')
                
                else :
                    global_.extend(f'\n{word["text"]}')

            global_.extend(" ")
            
        global_ = "".join(global_)
        texts_.append(global_)

    return texts_

In [175]:
cc = extract_text_from_bale_lines(words)

In [211]:
re.split(r"(\(.\) .*)\n",cc[69])

['Note: Basel III revisions published in December 2017 affect parts of this publication. https://www.bis.org/bcbs/publ/d424.htm \n2. Exposure measure \n',
 '(i) General measurement principles ',
 '157. The exposure measure for the leverage ratio should generally follow the accounting measure of exposure. To be measured consistently with financial accounts, the following should apply: \n\uf0b7 on-balance sheet, non-derivative exposures are net of specific provisions and valuation adjustments (eg credit valuation adjustments); \n\uf0b7 physical or financial collateral, guarantees or credit risk mitigation purchased is not allowed to reduce on-balance sheet exposures; and \n\uf0b7 netting of loans and deposits is not allowed. \n(ii) On-balance sheet items \n158. Banks should include items using their accounting balance sheet for the purposes of the leverage ratio. In addition, the exposure measure should include the following treatments for Securities Financing Transactions (SFT)54 and de

# IFRS Template

In [ ]:
text = global_process_ifrs("data/raw/IFRS_7.pdf")

In [ ]:
for j in range(len(text)) :
    text[j] = re.sub(r'\n.*\[Deleted\].*\n',"\n",text[j])
    text[j] = re.sub(r'\n.*\[deleted\].*\n',"\n",text[j])
final_text = "\n".join(text)
#If a part isn't finished at the end of a page
final_text = re.sub(r"([a-z]+)\n([a-z]+)", r"\1 \2",final_text)
#If a part isn't finished at the end of a page, while there is a footnote between the two 
final_text = re.sub(r"([a-z])\s*\n(_footnote_.*\n)([a-z].*\n)", r"\1\3\2",final_text)

final_text = re.sub(r"\.\.\.continued|\n_footnote_continued\.\.\.\n","", final_text)
final_text = re.sub(r"\n{2,}","\n", final_text)

In [ ]:
with open("data/cleaned/IFRS_7.txt", "w", encoding="utf-8") as file :
    file.write(final_text)